In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv1d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.conv3 = nn.Conv1d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm1d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10, num_ch=3):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # Calculate padding for input size 36
        padding_36 = 1  # For kernel_size=3, stride=1, padding=1
        self.conv1 = nn.Conv1d(num_ch, 64, kernel_size=3,
                               stride=1, padding=padding_36, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool1d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return torch.squeeze(out)


def ResNet18(num_ch=3, num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes, num_ch=num_ch)


def ResNet34(num_ch=3, num_classes=10):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=num_classes, num_ch=num_ch)


def ResNet50(num_ch=3, num_classes=10):
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes=num_classes, num_ch=num_ch)


def ResNet101(num_ch=3, num_classes=10):
    return ResNet(Bottleneck, [3, 4, 23, 3], num_classes=num_classes, num_ch=num_ch)


def ResNet152(num_ch=3, num_classes=10):
    return ResNet(Bottleneck, [3, 8, 36, 3], num_classes=num_classes, num_ch=num_ch)




In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def swish(x):
    return x * x.sigmoid()


def drop_connect(x, drop_ratio):
    keep_ratio = 1.0 - drop_ratio
    mask = torch.empty([x.shape[0], 1, 1], dtype=x.dtype, device=x.device)
    mask.bernoulli_(keep_ratio)
    x.div_(keep_ratio)
    x.mul_(mask)
    return x


class SE(nn.Module):
    '''Squeeze-and-Excitation block with Swish.'''

    def __init__(self, in_channels, se_channels):
        super(SE, self).__init__()
        self.se1 = nn.Conv1d(in_channels, se_channels, kernel_size=1, bias=True)
        self.se2 = nn.Conv1d(se_channels, in_channels, kernel_size=1, bias=True)

    def forward(self, x):
        out = F.adaptive_avg_pool1d(x, 1)
        out = swish(self.se1(out))
        out = self.se2(out).sigmoid()
        out = x * out
        return out


class Block(nn.Module):
    '''expansion + depthwise + pointwise + squeeze-excitation'''

    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride,
                 expand_ratio=1,
                 se_ratio=0.,
                 drop_rate=0.):
        super(Block, self).__init__()
        self.stride = stride
        self.drop_rate = drop_rate
        self.expand_ratio = expand_ratio

        # Expansion
        channels = expand_ratio * in_channels
        self.conv1 = nn.Conv1d(in_channels,
                               channels,
                               kernel_size=1,
                               stride=1,
                               padding=0,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(channels)

        # Depthwise conv
        self.conv2 = nn.Conv1d(channels,
                               channels,
                               kernel_size=kernel_size,
                               stride=stride,
                               padding=(1 if kernel_size == 3 else 2),
                               groups=channels,
                               bias=False)
        self.bn2 = nn.BatchNorm1d(channels)

        # SE layers
        se_channels = int(in_channels * se_ratio)
        self.se = SE(channels, se_channels)

        # Output
        self.conv3 = nn.Conv1d(channels,
                               out_channels,
                               kernel_size=1,
                               stride=1,
                               padding=0,
                               bias=False)
        self.bn3 = nn.BatchNorm1d(out_channels)

        # Skip connection if in and out shapes are the same (MV-V2 style)
        self.has_skip = (stride == 1) and (in_channels == out_channels)

    def forward(self, x):
        out = x if self.expand_ratio == 1 else swish(self.bn1(self.conv1(x)))
        out = swish(self.bn2(self.conv2(out)))
        out = self.se(out)
        out = self.bn3(self.conv3(out))
        if self.has_skip:
            if self.training and self.drop_rate > 0:
                out = drop_connect(out, self.drop_rate)
            out = out + x
        return out


class EfficientNet(nn.Module):
    def __init__(self, cfg, num_ch=3, num_classes=10):
        super(EfficientNet, self).__init__()
        self.cfg = cfg

        # Modify the first convolutional layer for input size [batch_size, num_channels, 36]
        self.conv1 = nn.Conv1d(num_ch,
                               32,
                               kernel_size=3,
                               stride=1,
                               padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm1d(32)
        self.layers = self._make_layers(in_channels=32)
        self.linear = nn.Linear(cfg['out_channels'][-1], num_classes)

    def _make_layers(self, in_channels):
        layers = []
        cfg = [self.cfg[k] for k in ['expansion', 'out_channels', 'num_blocks', 'kernel_size',
                                     'stride']]
        b = 0
        blocks = sum(self.cfg['num_blocks'])
        for expansion, out_channels, num_blocks, kernel_size, stride in zip(*cfg):
            strides = [stride] + [1] * (num_blocks - 1)
            for stride in strides:
                drop_rate = self.cfg['drop_connect_rate'] * b / blocks
                layers.append(
                    Block(in_channels,
                          out_channels,
                          kernel_size,
                          stride,
                          expansion,
                          se_ratio=0.25,
                          drop_rate=drop_rate))
                in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = swish(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = F.adaptive_avg_pool1d(out, 1)
        out = out.view(out.size(0), -1)
        dropout_rate = self.cfg['dropout_rate']
        if self.training and dropout_rate > 0:
            out = F.dropout(out, p=dropout_rate)
        out = self.linear(out)
        return out


def EfficientNetB0(num_ch=3, num_classes=10):
    cfg = {
        'num_blocks': [1, 2, 2, 3, 3, 4, 1],
        'expansion': [1, 6, 6, 6, 6, 6, 6],
        'out_channels': [16, 24, 40, 80, 112, 192, 320],
        'kernel_size': [3, 3, 5, 3, 5, 5, 3],
        'stride': [1, 2, 2, 2, 1, 2, 1],
        'dropout_rate': 0.2,
        'drop_connect_rate': 0.2,
    }
    return EfficientNet(cfg, num_classes=num_classes, num_ch=num_ch)


# Example usage:
# net = EfficientNetB0(num_ch=1, num_classes=10)
# x = torch.randn(2, 1, 36)
# y = net(x)
# print(y.shape)


In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

import warnings

warnings.filterwarnings(
    "ignore", message="Setting attributes on ParameterList is not supported."
)

cfg = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "VGG19": [ 64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M",],
}

class VGG1D(nn.Module):
    def __init__(
        self,
        vgg_name,
        num_ch=3,
        num_classes=10,
        bias=True,
        batch_norm=True,
        pooling="max",
        pooling_size=2,
        param_list=False,
        width_factor=1,
        stride=2,
    ):
        super(VGG1D, self).__init__()
        if pooling == True:
            pooling = "max"
        self.features = self._make_layers(
            cfg[vgg_name],
            ch=num_ch,
            bias=bias,
            bn=batch_norm,
            pooling=pooling,
            ps=pooling_size,
            param_list=param_list,
            width_factor=width_factor,
            stride=stride
        )
        stride_factor = 729 if stride == 1 else 1
        self.classifier = nn.Linear(int(512 * width_factor) * stride_factor, num_classes)

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg, ch, bias, bn, pooling, ps, param_list, width_factor, stride):
        layers = []
        in_channels = ch
        # Calculate padding for input size 36
        padding_36 = max(0, ((36 - 1) * stride - 32 + ps - 1) // 2)
        if ch == 1:
            layers.append(nn.ZeroPad1d(padding_36))
        if param_list:
            convLayer = Conv1dList
        else:
            convLayer = nn.Conv1d
        for x in cfg:
            if x == "M":
                if pooling == "max":
                    layers += [
                        nn.MaxPool1d(
                            kernel_size=ps, stride=stride, padding=ps // 2 + ps % 2 - 1
                        )
                    ]
                elif pooling == "avg":
                    layers += [
                        nn.AvgPool1d(
                            kernel_size=ps, stride=stride, padding=ps // 2 + ps % 2 - 1
                        )
                    ]
                else:
                    layers += [SubSampling1D(kernel_size=ps, stride=stride)]
            else:
                x = int(x * width_factor)
                if bn:
                    layers += [
                        convLayer(in_channels, x, kernel_size=3, padding=1, bias=bias),
                        nn.BatchNorm1d(x),
                        nn.ReLU(inplace=True),
                    ]
                else:
                    layers += [
                        convLayer(in_channels, x, kernel_size=3, padding=1),
                        nn.ReLU(inplace=True),
                    ]
                in_channels = x
        return nn.Sequential(*layers)


class Conv1dList(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        bias=True,
        padding_mode="constant",
    ):
        super().__init__()

        weight = torch.empty(out_channels, in_channels, kernel_size)

        nn.init.kaiming_uniform_(weight, a=math.sqrt(5))

        if bias is not None:
            bias = nn.Parameter(
                torch.empty(
                    out_channels,
                )
            )
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(bias, -bound, bound)

        n = max(1, 128 * 256 // (in_channels * kernel_size)) # * kernel_size
        weight = nn.ParameterList(
            [nn.Parameter(weight[j : j + n]) for j in range(0, len(weight), n)]
        )

        setattr(self, "weight", weight)
        setattr(self, "bias", bias)

        self.padding = padding
        self.padding_mode = padding_mode
        self.stride = stride

    def forward(self, x):

        weight = self.weight
        if isinstance(weight, nn.ParameterList):
            weight = torch.cat(list(self.weight))

        return F.conv1d(x, weight, self.bias, self.stride, self.padding)


class SubSampling1D(nn.Module):
    def __init__(self, kernel_size, stride=None):
        super(SubSampling1D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride if (stride is not None) else kernel_size

    def forward(self, input: torch.Tensor) -> torch.Tensor:
        return input.unfold(2, self.kernel_size, self.stride)[..., 0]




In [13]:
# Example usage:
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
net = ResNet18()#VGG1D('VGG11')#EfficientNetB0()


In [14]:
nodes, _ = get_graph_node_names(net)

In [15]:
nodes

['x',
 'conv1',
 'bn1',
 'relu',
 'layer1.0.conv1',
 'layer1.0.bn1',
 'layer1.0.relu',
 'layer1.0.conv2',
 'layer1.0.bn2',
 'layer1.0.add',
 'layer1.0.relu_1',
 'layer1.1.conv1',
 'layer1.1.bn1',
 'layer1.1.relu',
 'layer1.1.conv2',
 'layer1.1.bn2',
 'layer1.1.add',
 'layer1.1.relu_1',
 'layer2.0.conv1',
 'layer2.0.bn1',
 'layer2.0.relu',
 'layer2.0.conv2',
 'layer2.0.bn2',
 'layer2.0.shortcut.0',
 'layer2.0.shortcut.1',
 'layer2.0.add',
 'layer2.0.relu_1',
 'layer2.1.conv1',
 'layer2.1.bn1',
 'layer2.1.relu',
 'layer2.1.conv2',
 'layer2.1.bn2',
 'layer2.1.add',
 'layer2.1.relu_1',
 'layer3.0.conv1',
 'layer3.0.bn1',
 'layer3.0.relu',
 'layer3.0.conv2',
 'layer3.0.bn2',
 'layer3.0.shortcut.0',
 'layer3.0.shortcut.1',
 'layer3.0.add',
 'layer3.0.relu_1',
 'layer3.1.conv1',
 'layer3.1.bn1',
 'layer3.1.relu',
 'layer3.1.conv2',
 'layer3.1.bn2',
 'layer3.1.add',
 'layer3.1.relu_1',
 'layer4.0.conv1',
 'layer4.0.bn1',
 'layer4.0.relu',
 'layer4.0.conv2',
 'layer4.0.bn2',
 'layer4.0.shortcut

In [ ]:
get_graph_node_names